In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
start_time = pd.datetime.now()

In [2]:
df = pd.read_excel('card transactions.xlsx')

## I. Data Preprocessing

In [3]:
df.shape

(96753, 10)

In [4]:
df.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [5]:
df.dtypes

Recnum                        int64
Cardnum                       int64
Date                 datetime64[ns]
Merchnum                     object
Merch description            object
Merch state                  object
Merch zip                   float64
Transtype                    object
Amount                      float64
Fraud                         int64
dtype: object

In [6]:
## display null values and zeros; note there are different dtypes
## Don't worry about the warning, it's just a disagreement b/w numpy and native python

pd.DataFrame(dict(zip(['# of null', '# of zeros'], [df.isna().sum(), ((df == 0) | (df == '0')).sum()])))

/Users/aslanshi/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,# of null,# of zeros
Recnum,0,0
Cardnum,0,0
Date,0,0
Merchnum,3375,231
Merch description,0,0
Merch state,1195,0
Merch zip,4656,0
Transtype,0,0
Amount,0,0
Fraud,0,95694


According to the domain expert (aka our professor), for creating Benford's Law variables, we could ignore missing entries and zero entries in the two entity groups, which are "Merchnum" and "Cardnum". Also, transactions made in "FedEx" could also be ignored. Lastly for the entire project, we'll only focus on purchasing type "P" and exclude on outlier that have unresonable amount. So I will do some preprocessing here.

#### 1. Transaction type & outlier

In [7]:
df.Transtype.value_counts().to_frame()

,Transtype
P,96398
A,181
D,173
Y,1


In [8]:
df = df[df.Transtype == 'P']

In [9]:
df.sort_values('Amount', ascending = False).head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
52714,52715,5142189135,2010-07-13,NaN,INTERMEXICO,NaN,NaN,P,3102045.53,0
47339,47340,5142275225,2010-06-22,NaN,INTERSOL,NaN,NaN,P,47900.00,0
59516,59517,5142138135,2010-08-03,900009045549,KRACKELER SCIENTIFIC,NY,12202.0,P,30372.46,1
80886,80887,5142152857,2010-10-11,NaN,CANYON ELECTRIC,NV,89102.0,P,28392.84,1
89673,89674,5142183210,2010-11-29,NaN,CRUCIAL TECH./MICRON,ID,83642.0,P,27218.00,0


In [10]:
df = df[df.Amount < 3102045.53]

In [11]:
df.shape

(96397, 10)

## II. Risk Table Variables

In [12]:
## to be safe, check the data type of dates first

df.Date.dtypes

## good

dtype('<M8[ns]')

In [13]:
## find the day of the week

df['Dow'] = df.Date.apply(lambda x: calendar.day_name[x.weekday()])

In [14]:
## we only want to use data except oot

train_test = df[df.Date < '2016-11-01']

c = 4; nmid = 20; y_avg = train_test['Fraud'].mean()
y_dow = train_test.groupby('Dow')['Fraud'].mean()
num = train_test.groupby('Dow').size()
y_dow_smooth = y_avg + (y_dow - y_avg)/(1 + np.exp(-(num - nmid)/c))
df['Dow_Risk'] = df.Dow.map(y_dow_smooth)

In [15]:
df.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,Dow,Dow_Risk
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,Friday,0.022419
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,Friday,0.022419
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,Friday,0.022419
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,Friday,0.022419
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,Friday,0.022419


In [16]:
## risk table

y_dow_smooth.reset_index(name = 'Risk')

,Dow,Risk
0,Friday,0.022419
1,Monday,0.008749
2,Saturday,0.010810
3,Sunday,0.010444
4,Thursday,0.018797
5,Tuesday,0.007964
6,Wednesday,0.009530


## III. Filling in Missing Values

Before building other varibles, I need to deal with missing values.

In [17]:
pd.DataFrame(dict(zip(['# of null', '# of zeros'], [df.isna().sum(), ((df == 0) | (df == '0')).sum()])))

,# of null,# of zeros
Recnum,0,0
Cardnum,0,0
Date,0,0
Merchnum,3198,53
Merch description,0,0
Merch state,1020,0
Merch zip,4300,0
Transtype,0,0
Amount,0,0
Fraud,0,95338


### 3.1 Merchnum

This is hard for us to fill in since we aren't able to determine how they were designed. So I treat them as frivolous and replace them with record number.

In [18]:
df.loc[(df.Merchnum.isna()) | (df.Merchnum == '0'), 'Merchnum'] \
                = df.loc[(df.Merchnum.isna()) | (df.Merchnum == '0'), 'Recnum']

In [19]:
df.Merchnum = df.Merchnum.astype(str)

### 3.2 State & Zip

After some exploration, I found that for some records that miss state, or zip, or both, the associated merch desciption have such information in the dataset somewhere. So I would try to replace them with the most frequent. If there is no info, I will leave them as NaN.

**Important:** In real world practice, I should take validation set out and work on train/test set only.

In [20]:
df['Merch zip'] = df['Merch zip'].astype('Int64')

In [21]:
new = df.copy()

In [22]:
## value_counts() will not count NaN

def inital_fill(x):
    
    mer = x[0]
    state = x[1]
    zipcode = x[2]
    
    na = pd.isnull(state) + pd.isnull(zipcode)
    
    if na == 2:
        temp = df[df['Merch description'] == mer]
        states = temp['Merch state'].value_counts()
        zips = temp['Merch zip'].value_counts()
        try:
            mf_state = states.idxmax()
        except:
            mf_state = state
        try:
            mf_zip = zips.idxmax()
        except:
            mf_zip = zipcode
        return [mf_state, mf_zip]
    
    elif pd.isnull(state):
        temp = df[df['Merch description'] == mer]
        states = temp['Merch state'].value_counts()
        try:
            mf_state = states.idxmax()
        except:
            mf_state = state
        return [mf_state, zipcode]

    elif pd.isnull(zipcode):
        temp = df[df['Merch description'] == mer]
        zips = temp['Merch zip'].value_counts()
        try:
            mf_zip = zips.idxmax()
        except:
            mf_zip = zipcode
        return [state, mf_zip]
    
    else:
        return [state, zipcode]

In [23]:
new[['Merch state', 'Merch zip']] = new[['Merch description', 'Merch state', 'Merch zip']]\
                                    .apply(lambda x: pd.Series(inital_fill(x)), axis = 1)                             

In [24]:
new['Merch state'].isna().sum(), new['Merch zip'].isna().sum()

(363, 2043)

Now, for the remaining missing records (that miss in state or zip or both), we could not find information using their merchant description in the dataset.

For those not missing both, I found a nice library called **uszipcode**, which could return zip given state, and ves versa. Note, there are 1 or 2 or 3 digit zipcodes, and are also some zipcodes that dont't seem to be valid. Also, there are states that don't seem to be valid. I will exclude them in the following function and deal with them later.

The documentary page of the library is here: https://pypi.org/project/uszipcode/

In [25]:
## there are some states that are not valid us states...

new[~(new['Merch state'].isna()) & (new['Merch zip'].isna())]['Merch state'].unique()

array(['NY', 'GA', 'MA', 'OK', 'MD', 'BC', 'KY', 'CA', 'TX', 'QC', 'MO',
       'PA', 'ON', 'VA', 'MS', 'IL', 'NC', 'WV', 'FL', 'KS', 'CO', 'MB',
       'OR', 'WA', 'CT', 'WI', 'AZ', 'DC', 'IN', 'IA', 'MI', 'NJ', 'PQ',
       'MN', 'AL', 'OH', 'RI', 'MT', 'UT', 'NS', 'AK', 'NV', 'AB', 'NM',
       'LA', 'ID', 'ND', 'NE'], dtype=object)

In [26]:
## I use webscrapping to create a list of valid US abbrv.

from bs4 import BeautifulSoup
from urllib.request import urlopen

In [27]:
html = urlopen("https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations")
soup = BeautifulSoup(html)

In [28]:
table = soup.find("table", {'class':'wikitable sortable'})

In [29]:
names = table.find_all("span", {'class': 'monospaced'})
names = names[1:-15]

In [30]:
state_list = []
for name in names:
    name = name.get_text()[-2:]
    state_list.append(name)

In [31]:
from uszipcode import SearchEngine
searcher = SearchEngine()

In [32]:
new1 = new.copy()

In [33]:
def state_zip_fill(x):
    
    state = x[0]
    zipcode = x[1]
    
    na = pd.isnull(state) + pd.isnull(zipcode)
    
    if na == 1:
        if pd.isnull(state):
            result = searcher.by_zipcode(str(int(zipcode))).state
            if result == None:
                return [state, zipcode]
            else:
                return [result, zipcode]
        else:
            if state not in state_list:
                return [state, zipcode]
            else:
                result = searcher.by_state(state)[0].zipcode
                if result == None:
                    return [state, zipcode]
                else:
                    return [state, result]
    else:
        return [state, zipcode]

In [34]:
new1[['Merch state', 'Merch zip']] = new1[['Merch state', 'Merch zip']]\
                                    .apply(lambda x: pd.Series(state_zip_fill(x)), axis = 1) 

In [35]:
new1['Merch state'].isna().sum(), new1['Merch zip'].isna().sum()

(310, 454)

In [36]:
## There are 270 records that miss both entities information at this point, and there is no valuable information
## that we could use in the dataset; one of the approach I came up with is to find the associated cardnum and 
## replace the null with the most frequent value used by that card
## I am NOT 100% sure about this but unless there is better approach I will use this.

len(new1[(new1['Merch state'].isna()) & (new1['Merch zip'].isna())])

269

In [37]:
## I did some research and found that these are likely to be province codes of Canda
## I decide to manully set these zipcodes to be the state, since it's likely that when new data come in,
## these zip codes would be left as null as well, so just make them state as well.

new1[(new1['Merch zip'].isna()) & ~(new1['Merch state'].isna())]['Merch state'].unique()

array(['BC', 'QC', 'ON', 'MB', 'PQ', 'NS', 'AB'], dtype=object)

In [38]:
new1[~(new1['Merch zip'].isna()) & (new1['Merch state'].isna())]['Merch zip'].unique()

array([1400.0, 65132.0, 86899.0, 23080.0, 60528.0, 6.0, 8.0, 9.0, 50823.0,
       2.0, 48700.0, 1.0, 3.0, 801.0], dtype=object)

In [39]:
new2 = new1.copy()

In [40]:
## for the state above, I will first try to find records associated with them in the whole dataset
## if there is any, I will replace with the most frequent
## note, this time the granularity is different, records are not sorted by merchant descrip but by state

def zip_fill(x):
    
    state = x[0]
    zipcode = x[1]
    
    na = pd.isnull(state) + pd.isnull(zipcode)
    
    if na == 1 and pd.isnull(zipcode):
        temp = new2[new2['Merch state'] == state]
        zips = temp['Merch zip'].value_counts()
        try:
            mf_zip = zips.idxmax()
            return [state, mf_zip]
        except:
            return [state, state]
    else:
        return [state, zipcode]

In [41]:
new2[['Merch state', 'Merch zip']] = new2[['Merch state', 'Merch zip']]\
                                    .apply(lambda x: pd.Series(zip_fill(x)), axis = 1) 

In [42]:
## Similarly, I will first try to find records associated with the zip codes in the whole dataset
## if there is any, I will replace with the most frequent
## else, I will manually assign the state as string zipcode

def state_fill(x):
    
    state = x[0]
    zipcode = x[1]
    
    na = pd.isnull(state) + pd.isnull(zipcode)
    
    if na == 1 and pd.isnull(state):
        temp = new2[new2['Merch zip'] == zipcode]
        states = temp['Merch state'].value_counts()
        try:
            mf_state = states.idxmax()
            return [mf_state, zipcode]
        except:
            return [str(int(zipcode)), zipcode]
    else:
        return [state, zipcode]

In [43]:
new2[['Merch state', 'Merch zip']] = new2[['Merch state', 'Merch zip']]\
                                    .apply(lambda x: pd.Series(state_fill(x)), axis = 1) 

In [44]:
## if the above approach went successful, the following 3 numbers should all = 270
## which is the final remaining number of records that
##   1. their merchant descriptioon provide no information on state and zip in the whole dataset
##   2. they miss both state and zip code

len(new2[(new2['Merch state'].isna())]), len(new2[new2['Merch zip'].isna()]),\
                            len(new2[(new2['Merch state'].isna()) & (new2['Merch zip'].isna())])

(269, 269, 269)

In [45]:
## after checking with professor, the approach i will use is to find the associated cardnum and 
## replace the null with the most frequent value used by that card

new3 = new2.copy()

In [46]:
## if the cardnum group even don't have at least one non-null values, I have no choice but to set them as
## the most frequent of the whole dataset

def final_fill(x):
    
    temp = new3[new3.Cardnum == x]
    try:
        mf_state = temp['Merch state'].value_counts().idxmax()
    except:
        mf_state = new3['Merch state'].value_counts().idxmax()
    try:
        mf_zip = temp['Merch zip'].value_counts().idxmax()
    except:
        mf_zip = new3['Merch zip'].value_counts().idxmax()
        
    return pd.Series([mf_state, mf_zip])

In [47]:
nulls_df = new3[(new3['Merch state'].isna()) & (new3['Merch zip'].isna())]

null_merchs = nulls_df['Merch description'].unique()

for merch in null_merchs:
    
    temp = new3.loc[new3['Merch description'] == merch, 'Cardnum']
    result = temp.apply(final_fill).values
    new3.loc[new3['Merch description'] == merch, ['Merch state', 'Merch zip']] = result

In [48]:
## Finally.....

pd.DataFrame(dict(zip(['# of null', '# of zeros'], [new3.isna().sum(), ((new3 == 0) | (new3 == '0')).sum()])))

/Users/aslanshi/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,# of null,# of zeros
Recnum,0,0
Cardnum,0,0
Date,0,0
Merchnum,0,0
Merch description,0,0
Merch state,0,0
Merch zip,0,0
Transtype,0,0
Amount,0,0
Fraud,0,95338


In [49]:
new3.shape

(96397, 12)

In [50]:
new3.to_csv('cleaned_data.csv')

In [51]:
print('Duration: ', pd.datetime.now() - start_time)

Duration:  0:02:08.503877


In [65]:
new3[new3['Merch zip']=='AB']

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,Dow,Dow_Risk
44963,44964,5142117641,2010-06-15,44964,AGRICULTURE FOOD & RUR,AB,AB,P,37.00,0,Tuesday,0.007964
50776,50777,5142291592,2010-07-06,50777,SEQUITER SOFTWARE INC,AB,AB,P,752.83,0,Tuesday,0.007964
80132,80133,5142251947,2010-10-06,80133,TROUT UNLIMITED CANADA,AB,AB,P,225.00,0,Wednesday,0.009530
83155,83156,5142254784,2010-10-25,83156,ENVIRONMENTAL SERVICES AS,AB,AB,P,527.00,0,Monday,0.008749
89916,89917,5142111372,2010-11-30,89917,COMMUNICATION SYSTEMS,AB,AB,P,48.58,0,Tuesday,0.007964
